# Feeding data to a model 
    
 - Queue runners
 - Dataset(TFRecord)

### File I/O using queue runners 

 - Dataset Class는 Image read 기능을 직접적으로 제공하지 않기 때문에, 하위 레벨 API 인 `queue runner`를 이용한다. 
 - queue runners 는 일종의 thread runner 
 - Loading 과 Session이 병렬적으로 동작한다. $\rightarrow$ Loading Delay없이 연속적인 training이 가능하다. 
 - Directory 내 모든 학습 데이터(eg. Images)들을 직접 load하여 train할 경우 유용!
 

#### $\sharp$ 데이터를 학습 시키는 두 가지 방법 

 1 . Load $\rightarrow$ Preprocessing $\rightarrow$ train
     > raw data를 로드하여 전처리를 수행 한 후 학습 
 
 2 . Load $\rightarrow$ Preprocessing $\rightarrow$ write datasets $\rightarrow$ load dataset $\rightarrow$ train
     > raw data를 미리 전처리하여 파일로 쓰고, 전처리된 데이터를 로드하여 학습

> - 전처리 라인 길면, 학습의 성능이 저하 될 수 있다. 
> - 일반적으로는 전처리 파이프라인을 별도로 운영하고, 전처리된 데이터를 활용하지만, Case by Case이다. 
> - 각 데이터 포맷 별 로 to tfrecord transformer를 구축하여 트레인 시스템 운영해보겠다. 


### File I/O using Dataset(TFRecord)

- Tensorflow에서 권장하는 record dataset 
- csv와 유사하게 Dataset 클래스로 쉽게 데이터를 로드 할 수 있다. 
- 실제로 file load는 한 번만 이루어지므로 효율적인 data read각 가능하다. 
- HDFS 활용 시 매우 효과적이다. 

## Implementation

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt 
import numpy as np 
import os 

tf.reset_default_graph()

image_dir = "./cnn_dataset/images"
age_dir = './cnn_dataset/age.csv'
# Dataset 디렉토리에 있는 파일들의 리스트를 저장 
imgname_list = [os.path.join(image_dir, name) for name in os.listdir(image_dir)]

imgname_queue = tf.train.string_input_producer(imgname_list, num_epochs=1, shuffle=False,)
agename_queue = tf.train.string_input_producer([age_dir], num_epochs=1,shuffle=False)

img_reader = tf.WholeFileReader()
age_reader = tf.TextLineReader()

# key_img:파일이름, values_img:데이터  return 
key_img, raw_img = img_reader.read(imgname_queue)
key_age, raw_age = age_reader.read(agename_queue)

decoded_img = tf.image.decode_png(raw_img)
decoded_age = tf.decode_csv(raw_age, [[0]])

# gray image reshaping 
mean_reduced_image = tf.reduce_mean(decoded_img, axis=-1)
reshaped_img = tf.reshape(mean_reduced_image, [-1])

/Users/MAC/.pyenv/versions/anaconda3-5.0.1/envs/python_ana/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
with tf.Session() as sess :
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    thread = tf.train.start_queue_runners(sess, coord)

#     sess.run(tf.global_variables_initializer())
#     tf.initialize_all_variables()
    
    face_train_dir = './cnn_dataset/face_train.tfrecord'
    face_test_dir = './cnn_dataset/face_test.tfrecord'
    
    train_writer = tf.python_io.TFRecordWriter(face_train_dir)
    test_writer = tf.python_io.TFRecordWriter(face_test_dir)
    
    for i in range(99999999):
        try :
            _age, _img, _key = sess.run([decoded_age, reshaped_img, key_img])
            
            example = tf.train.Example()
            example.features.feature['age'].int64_list.value.append(_age[0])
            example.features.feature['img'].int64_list.value.extend(_img.tolist())
            
            if i < 6000:
                train_writer.write(example.SerializeToString())
                if i % 500 == 0:
                    print('{} train data has been written'.format(i))
            else:
                test_writer.write(example.SerializeToString())
                if i % 500 == 0 :
                    print('{} test data has been written'.format(i))
        
        except tf.errors.OutOfRangeError:
            print('size of total dataset {}'.format(i))
            break
    
    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(thread)

0 train data has been written
500 train data has been written
1000 train data has been written
1500 train data has been written
2000 train data has been written
2500 train data has been written
3000 train data has been written
3500 train data has been written
4000 train data has been written
4500 train data has been written
5000 train data has been written
5500 train data has been written
6000 test data has been written
6500 test data has been written
7000 test data has been written
7500 test data has been written
size of total dataset 7533
